# QMNN Basic Tutorial

This notebook demonstrates the basic usage of Quantum Memory-Augmented Neural Networks (QMNN).

## Learning Objectives
- Understand QMNN architecture
- Create and train a basic QMNN model
- Explore quantum memory operations
- Compare with classical baselines
- Visualize quantum advantages

In [ ]:
# Install QMNN if not already installed
# !pip install -e .

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# QMNN imports
from qmnn import QMNN, QuantumMemory, QMNNTrainer
from qmnn import get_system_info, get_available_features

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("QMNN Tutorial - Basic Usage")
print("=" * 40)

## 1. System Information and Setup

In [ ]:
# Check system capabilities
system_info = get_system_info()
features = get_available_features()

print("System Information:")
print(f"- QMNN Version: {system_info['qmnn_version']}")
print(f"- PyTorch Version: {system_info['pytorch_version']}")
print(f"- CUDA Available: {system_info['cuda_available']}")
print(f"- Max Recommended Qubits: {system_info['recommended_config']['max_qubits']}")

print("\nAvailable Features:")
for feature, available in features.items():
    status = "✓" if available else "✗"
    print(f"- {feature}: {status}")

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"\nUsing device: {device}")

## 2. Understanding Quantum Memory

In [ ]:
# Create a quantum memory instance
memory = QuantumMemory(
    capacity=32,
    embedding_dim=16,
    max_qubits=8
)

print("Quantum Memory Created:")
print(f"- Capacity: {memory.effective_capacity}")
print(f"- Embedding Dimension: {memory.embedding_dim}")
print(f"- Address Qubits: {memory.qram.address_qubits}")
print(f"- Data Qubits: {memory.qram.max_data_qubits}")

# Store some example data
print("\nStoring example data...")
for i in range(5):
    key = np.random.randn(16)
    value = np.random.randn(16)
    address = memory.store_embedding(key, value)
    print(f"Stored item {i+1} at address {address}")

# Check memory usage
usage = memory.memory_usage()
print(f"\nMemory Usage: {usage:.2%}")

# Test retrieval
query = key + 0.1 * np.random.randn(16)  # Noisy version of last key
retrieved = memory.retrieve_embedding(query)
similarity = np.dot(value, retrieved) / (np.linalg.norm(value) * np.linalg.norm(retrieved))
print(f"Retrieval similarity: {similarity:.3f}")

## 3. Creating a QMNN Model

In [ ]:
# Define model parameters
config = {
    'input_dim': 10,
    'hidden_dim': 64,
    'output_dim': 3,
    'memory_capacity': 32,
    'memory_embedding_dim': 32,
    'n_quantum_layers': 2,
    'max_qubits': 8,
    'use_attention': True,
    'dropout': 0.1
}

# Create QMNN model
model = QMNN(**config).to(device)

print("QMNN Model Created:")
print(f"- Total Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Get quantum information
quantum_info = model.get_quantum_info()
print(f"- Quantum Parameters: {quantum_info['quantum_parameters']:,}")
print(f"- Classical Parameters: {quantum_info['classical_parameters']:,}")
print(f"- Quantum Ratio: {quantum_info['quantum_ratio']:.2%}")
print(f"- Memory Capacity: {quantum_info['memory_capacity']}")

print("\nModel Architecture:")
print(model)

## 4. Generating Training Data

In [ ]:
def generate_memory_task_data(n_samples=1000, seq_len=15, input_dim=10, n_classes=3):
    """
    Generate data for a memory-dependent classification task.
    The model needs to remember information from early in the sequence
    to classify the entire sequence correctly.
    """
    X = torch.randn(n_samples, seq_len, input_dim)
    y = torch.zeros(n_samples, seq_len, dtype=torch.long)
    
    for i in range(n_samples):
        # Create memory cue in first few positions
        memory_cue = torch.randint(0, n_classes, (1,)).item()
        X[i, 0, 0] = memory_cue  # Embed cue in first position
        
        # Classification depends on memory cue and current input
        for t in range(seq_len):
            if t < 3:  # Early positions
                y[i, t] = memory_cue
            else:  # Later positions depend on memory + current input
                current_signal = (X[i, t, 1] > 0).long().item()
                y[i, t] = (memory_cue + current_signal) % n_classes
    
    return X, y

# Generate training and test data
print("Generating memory task data...")
X_train, y_train = generate_memory_task_data(800, 15, config['input_dim'], config['output_dim'])
X_test, y_test = generate_memory_task_data(200, 15, config['input_dim'], config['output_dim'])

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

print(f"Training data: {X_train.shape}, {y_train.shape}")
print(f"Test data: {X_test.shape}, {y_test.shape}")

# Visualize a sample
plt.figure(figsize=(12, 4))
sample_idx = 0
plt.subplot(1, 2, 1)
plt.imshow(X_train[sample_idx].cpu().T, aspect='auto', cmap='viridis')
plt.title('Input Sequence (Features over Time)')
plt.xlabel('Time Step')
plt.ylabel('Feature Dimension')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.plot(y_train[sample_idx].cpu(), 'o-')
plt.title('Target Sequence')
plt.xlabel('Time Step')
plt.ylabel('Class')
plt.grid(True)

plt.tight_layout()
plt.show()

## 5. Training the QMNN Model

In [ ]:
# Create trainer
trainer = QMNNTrainer(
    model=model,
    learning_rate=1e-3,
    device=device
)

# Training loop
n_epochs = 20
batch_size = 32
train_losses = []
train_accuracies = []
test_accuracies = []
memory_usages = []

print("Training QMNN model...")
for epoch in tqdm(range(n_epochs)):
    # Training
    epoch_loss = 0
    epoch_acc = 0
    n_batches = 0
    
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        
        metrics = trainer.train_step(batch_X, batch_y)
        epoch_loss += metrics['loss']
        epoch_acc += metrics['accuracy']
        n_batches += 1
    
    train_losses.append(epoch_loss / n_batches)
    train_accuracies.append(epoch_acc / n_batches)
    
    # Validation
    test_metrics = trainer.validate(X_test, y_test)
    test_accuracies.append(test_metrics['accuracy'])
    
    # Memory usage
    memory_usage = model.quantum_memory.memory_usage()
    memory_usages.append(memory_usage)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch+1}/{n_epochs}:")
        print(f"  Train Loss: {train_losses[-1]:.4f}")
        print(f"  Train Acc: {train_accuracies[-1]:.4f}")
        print(f"  Test Acc: {test_accuracies[-1]:.4f}")
        print(f"  Memory Usage: {memory_usages[-1]:.2%}")

print("\nTraining completed!")

## 6. Analyzing Results

In [ ]:
# Plot training curves
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(memory_usages, label='Memory Usage')
plt.title('Quantum Memory Usage')
plt.xlabel('Epoch')
plt.ylabel('Usage Ratio')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

print(f"Final Test Accuracy: {test_accuracies[-1]:.4f}")
print(f"Final Memory Usage: {memory_usages[-1]:.2%}")

## 7. Quantum Memory Analysis

In [ ]:
# Analyze quantum memory behavior
model.eval()
with torch.no_grad():
    # Get model predictions and memory info
    sample_batch = X_test[:5]
    predictions, memory_info = model(sample_batch)
    
    print("Memory Analysis:")
    print(f"- Memory reads: {memory_info.get('memory_reads', 'N/A')}")
    print(f"- Memory writes: {memory_info.get('memory_writes', 'N/A')}")
    print(f"- Memory usage: {memory_info['memory_usage']:.2%}")
    
    # Get quantum information
    quantum_info = model.get_quantum_info()
    print(f"\nQuantum Components:")
    print(f"- Quantum layers: {quantum_info['n_quantum_layers']}")
    print(f"- Quantum parameters: {quantum_info['quantum_parameters']:,}")
    print(f"- Classical parameters: {quantum_info['classical_parameters']:,}")
    print(f"- Quantum advantage ratio: {quantum_info['quantum_ratio']:.2%}")

# Memory capacity analysis
capacity_info = model.quantum_memory.capacity_bound()
print(f"\nMemory Capacity Analysis:")
for key, value in capacity_info.items():
    print(f"- {key}: {value}")

## 8. Comparison with Classical Baseline

In [ ]:
# Create a classical LSTM baseline
class LSTMBaseline(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.lstm = torch.nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.classifier = torch.nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        return self.classifier(lstm_out)

# Train LSTM baseline
lstm_model = LSTMBaseline(config['input_dim'], config['hidden_dim'], config['output_dim']).to(device)
lstm_trainer = QMNNTrainer(lstm_model, learning_rate=1e-3, device=device)

print("Training LSTM baseline...")
lstm_accuracies = []

for epoch in tqdm(range(n_epochs)):
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i:i+batch_size]
        batch_y = y_train[i:i+batch_size]
        lstm_trainer.train_step(batch_X, batch_y)
    
    test_metrics = lstm_trainer.validate(X_test, y_test)
    lstm_accuracies.append(test_metrics['accuracy'])

# Compare results
plt.figure(figsize=(10, 6))
plt.plot(test_accuracies, label='QMNN', linewidth=2)
plt.plot(lstm_accuracies, label='LSTM Baseline', linewidth=2)
plt.title('QMNN vs Classical Baseline')
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy')
plt.grid(True)
plt.legend()
plt.show()

print(f"\nFinal Results:")
print(f"QMNN Test Accuracy: {test_accuracies[-1]:.4f}")
print(f"LSTM Test Accuracy: {lstm_accuracies[-1]:.4f}")
improvement = (test_accuracies[-1] - lstm_accuracies[-1]) / lstm_accuracies[-1] * 100
print(f"QMNN Improvement: {improvement:+.2f}%")

## 9. Conclusions

In this tutorial, we:

1. **Explored quantum memory**: Learned how QRAM stores and retrieves information using quantum superposition
2. **Built a QMNN model**: Created a hybrid quantum-classical neural network
3. **Trained on memory tasks**: Demonstrated QMNN's ability to handle memory-dependent problems
4. **Analyzed quantum components**: Examined the quantum vs classical parameter ratio
5. **Compared with baselines**: Showed potential quantum advantages

### Key Takeaways:
- QMNN combines classical neural networks with quantum memory for enhanced capacity
- Quantum memory can provide advantages for tasks requiring long-term memory
- The system automatically handles hardware constraints and optimization
- Performance depends on the specific task and quantum component configuration

### Next Steps:
- Try different memory tasks and configurations
- Explore quantum transformers and attention mechanisms
- Experiment with error correction and federated learning
- Test on real quantum hardware (when available)